In [ ]:
import numpy as np 
from datetime import datetime
import pandas as pd
from scipy.stats import norm 
from scipy.optimize import brentq
from scipy.interpolate import CubicSpline
from scipy.ndimage import gaussian_filter1d
import matplotlib.pyplot as plt 


#load data which has expiry 31st Oct with current market price 755.40 and current risk free rate is 4.15%
r = 0.00415
S = 755.40
today = "27-09-2025"
df = pd.read_csv('/Users/deepak/Personal Projects/Quant/Projects/Options-Trading-Model/meta_options.csv')



# Data Prep

In [ ]:
def time_till_expiry(todays_date:str, expiry_date:str):
    """Date in the form DD-MM-YYYY"""
    # Convert strings to datetime objects
    td = datetime.strptime(todays_date, "%d-%m-%Y")
    ed = datetime.strptime(todays_date, "%d-%m-%Y")
    return (ed-td).days

def data_cleaning(file_path:str):
    df = pd.read_csv(file_path)
    df.drop(['Contract name', 'Last trade date (GMT-4)',"% change", 'Volume', 'Open interest', "Change"], axis=1)
    df["price"] = (df["Bid"] + df["Ask"])/2

## BLACK-SCHOLES functions

In [ ]:
def European_options_pricer(S:float, K:float, T:int, r:float, vol:float):
    """
    HELPER FUNCTION
    S: Price of Underlying at t=0
    K: Strike Price
    T: Time till Expiry 
    r: Risk-free rate
    vol: Implied Volatility
    """
    expiry = T
    d1_denom = vol * np.sqrt(expiry)
    d1_num = np.log(S/K) + (r + 0.5 * vol**2)* expiry
    d1 = d1_num/d1_denom
    d2 = d1 - vol * np.sqrt(expiry)
    return d1, d2

def European_call_price(S:float, K:float, T:int, r:float, vol:float):
    """
    S: Price of Underlying at t=0
    K: Strike Price
    T: Time till Expiry 
    r: Risk-free rate
    vol: Implied Volatility
    """
    expiry = T
    d1, d2 = European_options_pricer(S, K, T, r, vol)
    return S * norm.cdf(d1) - K*np.exp(-r * expiry)*norm.cdf(d2)

def European_put_price(S, K, T, r, vol):
    expiry = T
    d1, d2 = European_options_pricer(S, K, T, r, vol)
    return K * np.exp(-r * expiry) * norm.cdf(-d2) - S*norm.cdf(-d1)

def price_to_vol(S:float, K:float, T:int, r:float, price:float, type:int, vol=0.8):
    """
    Function which calculates the implied volatility from the options price
    S: Price of Underlying at t=0
    K: Strike Price
    T: Time till Expiry 
    r: Risk-free rate
    price: Price of the option
    type: Enter 0 for call option and 1 for put option
    vol: Volatility initial estimate
    """
    # Function whose root is the implied volatility
    def f(vol, S, K, T, r, price):
        if type:
            return European_put_price(S, K, T, r, vol) - price
        else: 
            return European_call_price(S, K, T, r, vol) - price
    
    # Brent's method to find implied volatility
    iv = brentq(f, 0.001, 3, args=(S, K, T, r, price, type))
    return iv

In [ ]:
def IV_cubic(Strike:list, Price:list, type:int, T:int):
    """Creates a smooth IV across strikes
    Strike: List of strike prices
    Price: List of prices at the given strike
    type: 0 for call, 1 for put
    T: Time till expiry
    """
    IV_list = []
    for i in range(len(Strike)):
        #Convert Prices into IV's
        IV_list.append(price_to_vol(S, K=Strike[i], T=T, r=r, price=Price[i], type=type))
    
    IV_cubic_func = CubicSpline(Strike, IV_list)

    #Plot cubic spline in IV space
    IV_vals = IV_cubic_func(Strike)
    plt.plot(Strike, IV_cubic_func(Strike))
    plt.show()

    #Smooth the function using a Gaussian Kernel 
    IV_smooth = gaussian_filter1d(IV_vals, sigma=0.01)

    return IV_smooth   


In [ ]:
def RND(Strike:list, Price:list):
    """Returns the RND for the option"""
    #x_[i] - x_[i-1]
    h = Strike[1] - Strike[0]
    first_deriv = np.gradient(Price, h)
    second_deriv = np.gradient(first_deriv, h)

    #RND is given by. 
    plt.plot(Strike, second_deriv)
    plt.show()
    return second_deriv
